In [3]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_distances
import numpy as np
import torch
from sentence_transformers import util
import random # 랜덤 선택을 위해 추가
import json
from tqdm import tqdm

In [4]:
def read_json_file(file_path):

    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        return data
    except FileNotFoundError:
        print(f"오류: 파일을 찾을 수 없습니다 - {file_path}")
        return None
    except json.JSONDecodeError:
        print(f"오류: JSON 디코딩 오류 - {file_path}")
        return None
    except Exception as e:
        print(f"읽는 중 오류 발생 {file_path}: {e}")
        return None


# 파일 경로
train_file_path = "/home/infidea/rebirth-hjun/KRAG_2025/Dataset/korean_language_rag_V1.0_train.json"
dev_file_path = "/home/infidea/rebirth-hjun/KRAG_2025/Dataset/korean_language_rag_V1.0_dev.json"
test_file_path = "/home/infidea/rebirth-hjun/KRAG_2025/Dataset/korean_language_rag_V1.0_test.json"

# 함수 호출하여 데이터 읽기
train_json_data = read_json_file(train_file_path)
dev_json_data = read_json_file(dev_file_path)
test_json_data = read_json_file(test_file_path)

datasets_to_combine = []

datasets_to_combine.extend(train_json_data)
datasets_to_combine.extend(dev_json_data)
datasets_to_combine.extend(test_json_data)

In [5]:
# df = pd.read_list(datasets_to_combine)
df = pd.DataFrame(datasets_to_combine)

In [6]:
df['output'][0]

{'answer': '"우동이 불을 것 같아 걱정이다."가 옳다. \'붇다\'의 어간 끝 받침 \'ㄷ\'은 모음으로 시작하는 어미 앞에서 \'ㄹ\'로 바뀐다. 따라서 \'붇다\'에 관형형 어미 \'-(으)ㄹ\'이 결합하면 \'불을\'이 된다. 마찬가지로 \'깨닫다\'는 \'깨달을\', \'듣다\'는 \'들을\'으로 활용한다. 다만, \'곧다\', \'뜯다\' 등은 \'곧을\', \'뜯을\'과 같이 어간이 바뀌지 않는 형태로 활용한다.'}

In [7]:
df_queries = pd.read_csv("/home/infidea/rebirth-hjun/KRAG_2025_2/KRAG_2025/RAG/0715_kanana-1.5-8b-instruct-2505_generated_queries_train_dev_test.csv")
df_queries['input'] = df['input']

# 2. 사전 학습된 언어 모델 로드
# 한국어 포함 다국어 지원 모델을 사용합니다.
model = SentenceTransformer('nlpai-lab/KURE-v1')

# 3. 각 칼럼의 문장을 임베딩 벡터로 변환
query_embeddings = model.encode(df_queries['generated_queries'].tolist())
question_embeddings = model.encode(df_queries['question'].tolist())
all_embeddings_tensor = torch.tensor(query_embeddings)

# 저장할 파일 경로 설정
save_path = "all_embeddings_tensor.pt" # .pt 또는 .pth 확장자를 주로 사용합니다.

# 텐서 저장
torch.save(all_embeddings_tensor, save_path)
print(f"텐서가 {save_path}에 성공적으로 저장되었습니다.")


텐서가 all_embeddings_tensor.pt에 성공적으로 저장되었습니다.


In [6]:
loaded_embeddings_tensor = torch.load(save_path)

In [ ]:
# # 2. 모든 n에 대해 반복 실행
# for n in range(len(query_embeddings)):
#     print(f"--- 원본 쿼리 #{n}: {df_queries['generated_queries'][n]} ---")
    
#     # 현재 n번째 쿼리 벡터
#     query_vector_tensor = all_embeddings_tensor[n:n+1]
    
#     # 3. 전체 임베딩과 코사인 유사도 계산
#     cosine_scores = util.cos_sim(query_vector_tensor, all_embeddings_tensor)
    
#     # 4. 상위 11개 결과 추출 (자기 자신 포함 시 상위 12개)
#     # 전체 쿼리 수가 12개 미만일 경우를 대비해 min() 사용
#     k_value = min(12, len(all_embeddings_tensor))
#     top_results = torch.topk(cosine_scores, k=k_value)
    
#     # 5. 유사한 쿼리의 인덱스를 리스트에 수집 (자기 자신 제외)
#     similar_indices = []
#     for idx in top_results[1][0]:
#         if idx.item() != n:
#             similar_indices.append(idx.item())
            
#     # 6. 수집된 상위 쿼리들 중 5개를 랜덤으로 선택
#     # 선택할 쿼리 수가 5개 미만일 경우를 대비해 min() 사용
#     num_to_sample = min(5, len(similar_indices))
#     randomly_selected_indices = random.sample(similar_indices, k=num_to_sample)
    
#     # 7. 랜덤으로 선택된 쿼리들의 내용을 리스트로 가져오기
#     selected_queries_list = [df_queries['generated_queries'][i] for i in randomly_selected_indices]
    
#     # 8. 쿼리 내용들을 하나의 문자열로 join
#     # 각 쿼리를 줄바꿈(\n)으로 구분하여 합칩니다. 원하면 " " 등으로 변경 가능합니다.
#     joined_queries_string = "\n".join(selected_queries_list)
    
#     print(f"\n[랜덤 선택 및 조합된 5개 쿼리]")
#     print(joined_queries_string)
#     print("\n" + "-" * 50 + "\n")

--- 원본 쿼리 #0: ["부사 불 불이 아님 부사격 조사"] ---

[랜덤 선택 및 조합된 5개 쿼리]
["부사격 조사 '-면서' 용언 어미 결합"]
["북 부사격조사 생략"]
["조사 어미 어간 활용 아닌 아닌예요 아닌아요 아닌가요 아닌데요 부정표현 어미변화"]
["부사 곰곰이 생각해 보았다"]
["의존명사 문제 없이 부사격 조사 없이"]

--------------------------------------------------

--- 원본 쿼리 #1: ["의존명사 이목구비 한글맞춤법 오똑한 오똑한 코 표준어 규정"] ---

[랜덤 선택 및 조합된 5개 쿼리]
["의존명사 컨트롤 어문규정"]
["의존명사 디렉토리 디렉터리 한글맞춤법"]
["의존명사 타깃 타겟 한글맞춤법"]
["의존명사 또아리 또아리 또아리 표준어 표준어 규정"]
["의존명사 빼기 배기 한자어 표준어 규정"]

--------------------------------------------------

--- 원본 쿼리 #2: ["묘비 건립 시기 표현 어문규정"] ---

[랜덤 선택 및 조합된 5개 쿼리]
["'입사 년도' 표기법 어문규정"]
["표기 원칙 어문규정 맞춤법"]
["문장 부사어 날짜 표기 어문규정"]
["문장 부사어 날짜 표기 어문규정"]
["날짜 : 2024년 3월 1일 어문규범 날짜 표기"]

--------------------------------------------------

--- 원본 쿼리 #3: ["저장할까요 취소할까요 맞춤법 어문규정"] ---

[랜덤 선택 및 조합된 5개 쿼리]
["문장 부사어 조사 어미 어문규정 맞춤법"]
["어문규정 어문규범 부합 여부 문장 수정 이유"]
["문장 부호 띄어쓰기 표준어 규정"]
["어문규정 어휘선택 수막새 암기와 한글맞춤법"]
["표기 원칙 어문규정 맞춤법"]

--------------------------------------------------

--- 원본 쿼리 #4: ["조사 생략 가능 여부 에어컨

In [8]:
q_list = []
# 2. 모든 n에 대해 반복 실행
for n in tqdm(range(len(query_embeddings))):
    # print(f"--- 원본 쿼리 #{n}: {df_queries['generated_queries'][n]} ---")
    
    # 현재 쿼리의 'question_type' 확인
    current_q_type = df_queries['input'][n]['question_type']
    # print(f"(쿼리 타입: {current_q_type})")

    # 현재 n번째 쿼리 벡터
    query_vector_tensor = all_embeddings_tensor[n:n+1]
    
    # 3. 전체 임베딩과 코사인 유사도 계산
    cosine_scores = util.cos_sim(query_vector_tensor, all_embeddings_tensor)[0] # 1D 텐서로 변환

    # --- 코드 수정 시작 ---

    # 4. 동일한 타입의 쿼리만 고려하도록 마스킹(masking)
    # 다른 타입의 쿼리와 자기 자신의 점수를 -1로 만들어 topk에서 제외
    search_limit = 748
    for i in range(len(cosine_scores)):
        # 검색 범위를 벗어나거나(748 이상), 타입이 다르거나, 자기 자신인 경우 점수를 -1로 설정
        if i >= search_limit or df_queries['input'][i]['question_type'] != current_q_type or i == n:
            cosine_scores[i] = -1

    # 5. 상위 11개 결과 추출 (자기 자신은 이미 제외됨)
    # 동일 타입의 전체 쿼리 수를 세고, 그 중에서 최대 11개를 선택
    num_same_type_queries = (df_queries['input'].apply(lambda x: x['question_type']) == current_q_type).sum() - 1
    
    if num_same_type_queries <= 0:
        print("\n비교할 동일한 타입의 다른 쿼리가 없습니다.")
        print("\n" + "-" * 50 + "\n")
        continue

    k_value = min(5, num_same_type_queries)
    top_results = torch.topk(cosine_scores, k=k_value)
    
    # 6. 유사한 쿼리의 인덱스를 리스트에 수집 (top_results가 이미 인덱스를 담고 있음)
    similar_indices = top_results[1].tolist()
    
    # --- 코드 수정 끝 ---

    # 7. 수집된 상위 쿼리들 중 5개를 랜덤으로 선택
    # 선택할 쿼리 수가 5개 미만일 경우를 대비해 min() 사용
    num_to_sample = min(1, len(similar_indices))
    randomly_selected_indices = random.sample(similar_indices, k=num_to_sample)
    
    # 8. 랜덤으로 선택된 쿼리들의 내용을 리스트로 가져오기
    selected_queries_list = ["question:" + df['input'][i]['question'] + "\nanswer:" + df['output'][i]['answer'] for i in randomly_selected_indices]
    
    # 9. 쿼리 내용들을 하나의 문자열로 join
    joined_queries_string = "\n\n".join(selected_queries_list)
    
    # print(f"\n[동일 타입 내 랜덤 선택 및 조합된 {num_to_sample}개 쿼리]")
    # print(joined_queries_string)
    # print("\n" + "-" * 50 + "\n")
    
    
    prompt = (
    "다음은 질문 유형과 답변 형식을 이해하는 데 도움이 되는 여러 예시입니다. 이 예시들의 내용에 얽매이지 않고, 주어진 질문에 대해 가장 적절한 답변을 생성하세요.\n\n"
    f"{joined_queries_string}\n\n"
    "**위 'answer:' 뒤의 출력 형식을 엄격히 준수하여 답변을 생성하십시오.**\n"
    "\n이제 질문을 시작합니다.\n"
    f"question:{df['input'][n]['question']}"
    # f"{instruction}"
    )
    
    q_list.append(prompt)

100%|██████████| 1247/1247 [00:48<00:00, 25.77it/s]


In [13]:
answer_list = [] # 'answer' 변수명 중복을 피하기 위해 이름 변경

for item in datasets_to_combine:
    
    answer_value = item.get('output', {}).get('answer', '')
    answer_list.append(answer_value)

In [14]:
retrieval_df = pd.read_csv("/home/infidea/rebirth-hjun/KRAG_2025_2/KRAG_2025/RAG/0715-retrieval_results.csv")

r_q_list = []
print(len(q_list), len(retrieval_df))
 
for i in range(len(q_list)):
    r_q_list.append("다음 글은 어문 규범에 맞는 표현을 고르거나 어문 규범에 따라 문장을 교정하고 그 이유를 설명할 수 있는 자료이다\n\n" + retrieval_df['retrieved_content'][i] + "\n\n" + q_list[i])

1247 1247


In [15]:
print(r_q_list[0])

다음 글은 어문 규범에 맞는 표현을 고르거나 어문 규범에 따라 문장을 교정하고 그 이유를 설명할 수 있는 자료이다

- ㄴ: 불이 꺼져간다., 내 힘으로 막아낸다., 어머니를 도와드린다., 그릇을 깨뜨려버렸다.,비가 올듯하다., 그 일은 할만하다., 일이 될법하다., 비가 올성싶다., 잘 아는척한다. 
 
다만, 앞말에 조사가 붙거나 앞말이 합성 용언인 경우, 그리고 중간에 조사가 들어갈 때에는 그 뒤에 오는 보조 용언은 띄어 쓴다.

다음은 질문 유형과 답변 형식을 이해하는 데 도움이 되는 여러 예시입니다. 이 예시들의 내용에 얽매이지 않고, 주어진 질문에 대해 가장 적절한 답변을 생성하세요.

question:"표본 상자는 햇빛이 바로 {내리쬐이지/내리쪼이지} 않고 통풍이 잘 되는 건조한 곳에 두어야 한다." 가운데 올바른 것을 선택하고, 그 이유를 설명하세요.
answer:"표본 상자는 햇빛이 바로 내리쪼이지 않고 통풍이 잘 되는 건조한 곳에 두어야 한다."가 옳다. '볕이 들어 비치다'를 의미하는 단어는 '쪼이다'이다. '쪼이다' 대신 '쬐이다'를 쓰는 것은 국어의 일반적인 음운 현상으로 설명하기 어려우므로 표준어로 인정하지 않는다.

**위 'answer:' 뒤의 출력 형식을 엄격히 준수하여 답변을 생성하십시오.**

이제 질문을 시작합니다.
question:"우동이 {불을/불} 것 같아 걱정이다." 가운데 올바른 것을 선택하고, 그 이유를 설명하세요.


In [16]:
answer_list[0]

'"우동이 불을 것 같아 걱정이다."가 옳다. \'붇다\'의 어간 끝 받침 \'ㄷ\'은 모음으로 시작하는 어미 앞에서 \'ㄹ\'로 바뀐다. 따라서 \'붇다\'에 관형형 어미 \'-(으)ㄹ\'이 결합하면 \'불을\'이 된다. 마찬가지로 \'깨닫다\'는 \'깨달을\', \'듣다\'는 \'들을\'으로 활용한다. 다만, \'곧다\', \'뜯다\' 등은 \'곧을\', \'뜯을\'과 같이 어간이 바뀌지 않는 형태로 활용한다.'

In [17]:
df = pd.DataFrame({
    'question': r_q_list,
    'answer': answer_list
})

In [18]:
df.to_csv("0715-train_dev_test.csv", index=False)